<img alt="Colaboratory logo" width="15%" src="https://raw.githubusercontent.com/carlosfab/escola-data-science/master/img/novo_logo_bg_claro.png">

#### **Data Science na Prática 3.0**


---

# PREVISÃO DE DEMANDA

Como falamos antes, nossos templates ficarão cada vez mais simples!!

O objetivo desse projeto é prever a demanda por vinhos em lojas especializadas em vinho. Para esse projeto, temos 2 datasets, um sobre o catálogo de vinhos da loja e outro sobre as vendas.

* [Dados sobre os produtos](https://raw.githubusercontent.com/rafaelnduarte/sigmoidal_data/master/products.csv)
* [Dados de Vendas](https://raw.githubusercontent.com/rafaelnduarte/sigmoidal_data/master/sales-clean.csv)

Nesse projeto, é VITAL que haja MUITA ATENÇÃO sobre os detalhes mencionados que permeiam as análise de séries temporais. É um projeto que demanda muita atenção e cuidado, e uma documentação feita minuciosamente.

Aqui, queremos, como sempre, trazer o máximo de autonomia e independência pra vocês. Façam o download, e disponibilizem os seus dados, carreguem eles aqui, e desenvolvam o projeto.

## Objetivos

* Contextualização do problema, dos dados, do projeto.
* Análise completa dos dados e do problema (Na mão, ou com Pandas Profiling, SweetViz, etc)
* Desenvolver uma solução de Machine Learning eficiente com PyCaret para Classificação

LEMBREM-SE: Documentação, Storytelling, Artigo com Código! 

Mãos à obra e boa sorte!
